# Clustering Crypto

In [455]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
from pathlib import Path
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# ignore DataConversionWarning messages
import warnings
from sklearn.exceptions import DataConversionWarning


### Fetching Cryptocurrency Data

In [456]:
# Read in CSV and create a DataFrame
cryptotocurrency_data = pd.read_csv("C:/Users/erahm/unit13-challenge/ClusteringCrypto/crypto_data.csv", header=None)
cryptotocurrency_data.columns = ["Blank", "CoinName", "Algorithm", "IsTrading", "ProofType", "TotalCoinsMined","TotalCoinSupply"]
cryptotocurrency_data.head()

,Blank,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,NaN,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
1,42,42 Coin,Scrypt,TRUE,PoW/PoS,41.99995383,42
2,365,365Coin,X11,TRUE,PoW/PoS,NaN,2300000000
3,404,404Coin,Scrypt,TRUE,PoW/PoS,1055184902,532000000
4,611,SixEleven,SHA-256,TRUE,PoW,NaN,611000


In [457]:
# Drop first column and row of dataframe
new_crypto_currency_df = cryptotocurrency_data.drop('Blank', axis=1).drop(labels=0, axis=0)
new_crypto_currency_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
1,42 Coin,Scrypt,TRUE,PoW/PoS,41.99995383,42
2,365Coin,X11,TRUE,PoW/PoS,NaN,2300000000
3,404Coin,Scrypt,TRUE,PoW/PoS,1055184902,532000000
4,SixEleven,SHA-256,TRUE,PoW,NaN,611000
5,808,SHA-256,TRUE,PoW/PoS,0,0


In [458]:
# Check the data type of each column
new_crypto_currency_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1252 entries, 1 to 1252
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CoinName         1252 non-null   object
 1   Algorithm        1252 non-null   object
 2   IsTrading        1252 non-null   object
 3   ProofType        1252 non-null   object
 4   TotalCoinsMined  744 non-null    object
 5   TotalCoinSupply  1252 non-null   object
dtypes: object(6)
memory usage: 68.5+ KB


In [459]:
# Detect missing values by column and row for the crypto data
new_crypto_currency_df.isnull()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
1,False,False,False,False,False,False
2,False,False,False,False,True,False
3,False,False,False,False,False,False
4,False,False,False,False,True,False
5,False,False,False,False,False,False
...,...,...,...,...,...,...
1248,False,False,False,False,False,False
1249,False,False,False,False,False,False
1250,False,False,False,False,True,False
1251,False,False,False,False,True,False


In [460]:
# Determine the percentage of nulls by column for the crypto data
new_crypto_currency_df.isnull().mean() * 100

CoinName            0.00000
Algorithm           0.00000
IsTrading           0.00000
ProofType           0.00000
TotalCoinsMined    40.57508
TotalCoinSupply     0.00000
dtype: float64

In [461]:
# Find the total sum of nulls by column for the crypto data
new_crypto_currency_df.isnull().sum()

CoinName             0
Algorithm            0
IsTrading            0
ProofType            0
TotalCoinsMined    508
TotalCoinSupply      0
dtype: int64

In [462]:
# Cleanse nulls in 'TotalCoinsMined' column by filling with 'Unknown'
new_crypto_currency_df["TotalCoinsMined"] = new_crypto_currency_df["TotalCoinsMined"].fillna("Unknown")
new_crypto_currency_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
1,42 Coin,Scrypt,TRUE,PoW/PoS,41.99995383,42
2,365Coin,X11,TRUE,PoW/PoS,Unknown,2300000000
3,404Coin,Scrypt,TRUE,PoW/PoS,1055184902,532000000
4,SixEleven,SHA-256,TRUE,PoW,Unknown,611000
5,808,SHA-256,TRUE,PoW/PoS,0,0
...,...,...,...,...,...,...
1248,BitcoinPlus,Scrypt,TRUE,PoS,128326.9963,1000000
1249,DivotyCoin,Scrypt,FALSE,PoW/PoS,21491213.46,100000000
1250,Giotto Coin,Scrypt,FALSE,PoW/PoS,Unknown,233100000
1251,OpenSourceCoin,SHA-256,FALSE,PoW/PoS,Unknown,21000000


In [463]:
# Check again for the total sum of nulls by column for the crypto data.  
# There are no remaining null values
new_crypto_currency_df.isnull().sum()

CoinName           0
Algorithm          0
IsTrading          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

### Data Preprocessing

In [464]:
# Keep only cryptocurrencies that are trading
index_names = new_crypto_currency_df[ (new_crypto_currency_df['IsTrading'] == 'FALSE')].index
  
# Drop these given row indexes from dataFrame
new_crypto_currency_df.drop(index_names, inplace = True)

# Print dataframe 
new_crypto_currency_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
1,42 Coin,Scrypt,TRUE,PoW/PoS,41.99995383,42
2,365Coin,X11,TRUE,PoW/PoS,Unknown,2300000000
3,404Coin,Scrypt,TRUE,PoW/PoS,1055184902,532000000
4,SixEleven,SHA-256,TRUE,PoW,Unknown,611000
5,808,SHA-256,TRUE,PoW/PoS,0,0
...,...,...,...,...,...,...
1244,Super Zero,Ethash,TRUE,PoW,Unknown,1000000000
1245,UOS,SHA-256,TRUE,DPoI,Unknown,1000000000
1246,Beldex,CryptoNight,TRUE,PoW,980222595,1400222610
1247,Horizen,Equihash,TRUE,PoW,7296537.5,21000000


In [465]:
# Confirm that we've removed all cryptocurrencies that are not trading by counting the number of distinct values in the 'IsTrading' column
new_crypto_currency_df['IsTrading'].value_counts()

TRUE    1144
Name: IsTrading, dtype: int64

In [466]:
# Keep only cryptocurrencies with a working algorithm
# Based on the null function above, we know that there were no null values in the Algorithm column.

# First, we can print the number of unique values in the 'Algorithm column' by algorithm
new_crypto_currency_df['Algorithm'].value_counts(ascending=False)

Scrypt                   394
X11                      182
SHA-256                  121
X13                       54
PoS                       42
                        ... 
VeChainThor Authority      1
Ouroboros                  1
POS 2.0                    1
Proof-of-BibleHash         1
TRC10                      1
Name: Algorithm, Length: 89, dtype: int64

In [467]:
# We can also use the describe function to get descriptive statistics about the 'Algorithm' column.  We see that there are 89 unique algorithms in the 'Algorithms' column.
new_crypto_currency_df['Algorithm'].describe()

count       1144
unique        89
top       Scrypt
freq         394
Name: Algorithm, dtype: object

In [468]:
# Find the percentage of total valeus by algorithm
new_crypto_currency_df['Algorithm'].value_counts(normalize=True)

Scrypt                   0.344406
X11                      0.159091
SHA-256                  0.105769
X13                      0.047203
PoS                      0.036713
                           ...   
VeChainThor Authority    0.000874
Ouroboros                0.000874
POS 2.0                  0.000874
Proof-of-BibleHash       0.000874
TRC10                    0.000874
Name: Algorithm, Length: 89, dtype: float64

In [469]:
# Next, we can convert the Algorithm counts to a string to display all unique values in the Algorithm column. Since we had no null values and 
new_crypto_currency_df['Algorithm'].value_counts(normalize=True)
new_crypto_currency_df['Algorithm'].value_counts(normalize=True).to_string()

'Scrypt                    0.344406\nX11                       0.159091\nSHA-256                   0.105769\nX13                       0.047203\nPoS                       0.036713\nCryptoNight               0.029720\nQuark                     0.022727\nEquihash                  0.020105\nNIST5                     0.017483\nDPoS                      0.015734\nMultiple                  0.015734\nNeoScrypt                 0.014860\nSHA-256D                  0.013986\nEthash                    0.013986\nX15                       0.012238\nLyra2REv2                 0.006119\nLyra2RE                   0.005245\nQuBit                     0.005245\nBlake                     0.005245\nXEVAN                     0.005245\nScrypt-n                  0.005245\nCryptoNight-V7            0.004371\nLyra2Z                    0.004371\nGroestl                   0.004371\nDagger                    0.003497\nX16R                      0.003497\nSkein                     0.003497\nBlake2S                   0

Based on the .descibe() and .value_counts() functions, as well as the null analysis from above, we can see the breakdown of each algorithm by percentage and count. We were also able to confirm that all cryptocurrencies have a working algorithm.

In [470]:
# Remove the "IsTrading" column
crypto_currency_df = new_crypto_currency_df.drop('IsTrading', axis=1)
crypto_currency_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
1,42 Coin,Scrypt,PoW/PoS,41.99995383,42
2,365Coin,X11,PoW/PoS,Unknown,2300000000
3,404Coin,Scrypt,PoW/PoS,1055184902,532000000
4,SixEleven,SHA-256,PoW,Unknown,611000
5,808,SHA-256,PoW/PoS,0,0
...,...,...,...,...,...
1244,Super Zero,Ethash,PoW,Unknown,1000000000
1245,UOS,SHA-256,DPoI,Unknown,1000000000
1246,Beldex,CryptoNight,PoW,980222595,1400222610
1247,Horizen,Equihash,PoW,7296537.5,21000000


In [471]:
# Remove rows with cryptocurrencies having no coins mined

# Sort dataframe by 'TotalCoinsMind' to find the indexes that have '0' coins mined
# Note: There are 1144 unique rows
crypto_currency_df.sort_values(by=['TotalCoinsMined'])

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
1201,Fiii,SHA3-256,DPoC,-5917977548,5000000000
254,CyberCoin,Scrypt,PoW/PoS,0,0
362,AEON,CryptoNight-Lite,PoW,0,18400000
361,Universal Currency,SHA-256,PoW/PoS,0,210000000
1094,ETHER-1,Ethash,PoW,0,13666237
...,...,...,...,...,...
741,DuxCoin,Scrypt,PoW/PoS,Unknown,1680000000
742,PoisonIvyCoin,Scrypt,PoW/PoS,Unknown,4666666667
412,EvilCoin,X11,PoW/PoS,Unknown,21024000
736,MilkCoin,SHA-256,PoW/PoS,Unknown,4500000000


In [472]:
# Remove rows with cryptocurrencies having no coins mined

# Next, we can filter the dataframe to find the total number of rows where there are '0' coins mind
# Note: There are 152 rows with zero coins mined, so we should expect to see 992 rows remaining once we drop these rows
df_filtered = crypto_currency_df[(crypto_currency_df['TotalCoinsMined']=='0')]
print(df_filtered)

          CoinName    Algorithm ProofType TotalCoinsMined TotalCoinSupply
5              808      SHA-256   PoW/PoS               0               0
19        Bitstake          X11   PoW/PoS               0         1300000
30           ACoin      SHA-256       PoW               0         1600000
31       Aero Coin          X13       PoS               0         7000000
36        ApexCoin          X13   PoW/PoS               0         6000000
...            ...          ...       ...             ...             ...
1181   Quazar Coin  CryptoNight       PoW               0        18446744
1184      ParkByte      SHA-256   PoW/PoS               0        25000000
1198       Dotcoin       Scrypt       PoW               0       890000000
1200  The Hempcoin       Scrypt   PoW/PoS               0       300000000
1224        iOlite       Ethash       PoW               0      1000000000

[152 rows x 5 columns]


In [473]:
# Remove rows with cryptocurrencies having no coins mined

# Finally, we can drop the rows that have zero coins mined and verify that there are 992 rows remaining, with the 152 rows with zero coins mined removed
crypto_currency_df.drop(crypto_currency_df[crypto_currency_df['TotalCoinsMined'] == '0'].index, inplace = True)
crypto_currency_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
1,42 Coin,Scrypt,PoW/PoS,41.99995383,42
2,365Coin,X11,PoW/PoS,Unknown,2300000000
3,404Coin,Scrypt,PoW/PoS,1055184902,532000000
4,SixEleven,SHA-256,PoW,Unknown,611000
6,EliteCoin,X13,PoW/PoS,29279424623,3.14159E+11
...,...,...,...,...,...
1244,Super Zero,Ethash,PoW,Unknown,1000000000
1245,UOS,SHA-256,DPoI,Unknown,1000000000
1246,Beldex,CryptoNight,PoW,980222595,1400222610
1247,Horizen,Equihash,PoW,7296537.5,21000000


In [474]:
# Check for rows where there are 'N/A' text values by checking for 'N/A' values in any column. We can see from the output below that there are no N/A values in any column
crypto_currency_df[crypto_currency_df.isin(['n/a', 'na', 'N/A', 'NA', '--', '---']).any(axis=1)]

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply


In [475]:
# Drop rows from 'TotalCoinsMined' column where value = 'Unknown'
crypto_currency_df.drop(crypto_currency_df[crypto_currency_df['TotalCoinsMined'] == 'Unknown'].index, inplace = True)
crypto_currency_df

# Check for rows where there are 'Unknown' text values in the 'TotalCoinsMined' column
crypto_currency_df[crypto_currency_df.isin(['Unknown']).any(axis=1)]

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply


In [476]:
# Create new dataframe named coins_name to hold all the Coin Names and use the crypto_df.index as the index for this dataframe
coins_name = crypto_currency_df[['CoinName']].copy()
coins_name

,CoinName
1,42 Coin
3,404Coin
6,EliteCoin
8,Bitcoin
9,Ethereum
...,...
1239,ZEPHYR
1243,Gapcoin
1246,Beldex
1247,Horizen


In [477]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_currency_final_df = crypto_currency_df.drop('CoinName', axis=1)
crypto_currency_final_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
1,Scrypt,PoW/PoS,41.99995383,42
3,Scrypt,PoW/PoS,1055184902,532000000
6,X13,PoW/PoS,29279424623,3.14159E+11
8,SHA-256,PoW,17927175,21000000
9,Ethash,PoW,107684222.7,0
...,...,...,...,...
1239,SHA-256,DPoS,1999999995,2000000000
1243,Scrypt,PoW/PoS,14931046.15,250000000
1246,CryptoNight,PoW,980222595,1400222610
1247,Equihash,PoW,7296537.5,21000000


In [478]:
# Filter crypto_currency_final_df by 'TotalCoinsMined' and 'TotalCoinSupply' columns for additional data cleaning
filtered_df = crypto_currency_final_df.filter(['TotalCoinsMined','TotalCoinSupply'], axis=1)
filtered_df

# Convert 'TotalCoinsMined' and 'TotalCoinSupply' columns to floats
filtered_df = new.apply(pd.to_numeric, downcast='float')
filtered_df

# Check for data types across dataframe
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 533 entries, 1 to 1248
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TotalCoinsMined  533 non-null    float32
 1   TotalCoinSupply  533 non-null    float32
dtypes: float32(2)
memory usage: 24.5 KB


In [ ]:
# Merge old dataframe with new 'TotalCoinsMined' and 'TotalCoinSupply' columns that are now converted to floating point numbers
cc_final_df = filtered_df.merge(crypto_currency_final_df, left_index=True, right_index=True).drop(columns=['TotalCoinsMined_y', 'TotalCoinSupply_y'])
cc_final_df

,TotalCoinsMined_x,TotalCoinSupply_x,Algorithm,ProofType
1,4.199995e+01,4.200000e+01,Scrypt,PoW/PoS
3,1.055185e+09,5.320000e+08,Scrypt,PoW/PoS
6,2.927942e+10,3.141590e+11,X13,PoW/PoS
8,1.792718e+07,2.100000e+07,SHA-256,PoW
9,1.076842e+08,0.000000e+00,Ethash,PoW
...,...,...,...,...
1239,2.000000e+09,2.000000e+09,SHA-256,DPoS
1243,1.493105e+07,2.500000e+08,Scrypt,PoW/PoS
1246,9.802226e+08,1.400223e+09,CryptoNight,PoW
1247,7.296538e+06,2.100000e+07,Equihash,PoW


In [ ]:
# Create dummy variables for text features
# Encode the categorical features using one-hot encoding
X = pd.get_dummies(cc_final_df)
X

,TotalCoinsMined_x,TotalCoinSupply_x,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
1,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
6,2.927942e+10,3.141590e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
8,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1239,2.000000e+09,2.000000e+09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1243,1.493105e+07,2.500000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1246,9.802226e+08,1.400223e+09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1247,7.296538e+06,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Standardize data with StandardScaler
crypto_scaler = StandardScaler().fit_transform(X)
print(crypto_scaler[0:5])

[[-0.0433555 -0.0433555 -0.0433555 ... -0.0433555 -0.0433555 -0.0433555]
 [-0.0433555 -0.0433555 -0.0433555 ... -0.0433555 -0.0433555 -0.0433555]
 [-0.0433555 -0.0433555 -0.0433555 ... -0.0433555 -0.0433555 -0.0433555]
 [-0.0433555 -0.0433555 -0.0433555 ... -0.0433555 -0.0433555 -0.0433555]
 [-0.0433555 -0.0433555 -0.0433555 ... -0.0433555 -0.0433555 -0.0433555]]


### Reducing Dimensions Using PCA

Now that we have completed data cleaning, pre-processing and standardization, the next step is to reduce the number of total dimnensions using PCA.  We use, PCA or Principal Component Analysis, when the number of input features (or dimensions) is too high and we want to speed up our machine-learning algorithm.  PCA accomplishes this by transforming a large set of variables into a smaller subset that retains most of the information from the original set.  The two libraries that need to be imported from sklearn to use PCA are: StandardScaler and PCA, which are included above.

In [ ]:
# Use PCA to reduce dimensions to 3 principal components

# Since we standardized the features in the previous step, we now use PCA to reduce the number of features
# in the dataset. For this dataset, we are going to reduce the features/dimensions to 3 principal components.
# We did this by specifiying the number of features via the n_components parameter.

# Initialize PCA model
pca = PCA(n_components=3)

# Get 3 principal components for the crypto data
crypto_pca = pca.fit_transform(crypto_scaler)

Following our PCA dimensionality reduction, we now have a smaller set of features called principal components. The new components are simply three key dimensions of variation that include most of the information that was within the original dataset - simply put, a reduced representation of the original data.

Now, we need to transform our principal components into a DataFrame that can then be used to fit the k-means algorithm.

In [ ]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"])
pcs_df.head()

,PC 1,PC 2,PC 3
0,0.060281,-0.379949,-0.141217
1,0.047265,-0.222049,0.001138
2,-0.033991,-0.129254,-0.491881
3,-0.045938,-0.245716,-0.302986
4,-0.056947,-0.377645,0.107146


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [ ]:
# Finding the best value for k
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Creating the Elbow Curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

C:\Users\erahm\folder\envs\algotrading\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  f"KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [ ]:
# Predicting clusters with k=4

# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Add the predicted class columns
pcs_df["class"] = model.labels_
pcs_df.head()

,PC 1,PC 2,PC 3,class
0,0.060281,-0.379949,-0.141217,0
1,0.047265,-0.222049,0.001138,0
2,-0.033991,-0.129254,-0.491881,0
3,-0.045938,-0.245716,-0.302986,0
4,-0.056947,-0.377645,0.107146,0


In [481]:
# Merge all dataframes and create final dataframe called clustered_df

# Merge crypto_currency_final_df dataframe with pcs_df dataframe
mergedDf = cc_final_df.merge(pcs_df, left_index=True, right_index=True)

# Merge 'mergedDF' dataframe with coins_name dataframe
clustered_df = mergedDf.merge(coins_name, left_index=True, right_index=True)
clustered_df

,TotalCoinsMined_x,TotalCoinSupply_x,Algorithm,ProofType,PC 1,PC 2,PC 3,CoinName
1,4.199995e+01,4.200000e+01,Scrypt,PoW/PoS,0.047265,-0.222049,0.001138,42 Coin
3,1.055185e+09,5.320000e+08,Scrypt,PoW/PoS,-0.045938,-0.245716,-0.302986,404Coin
6,2.927942e+10,3.141590e+11,X13,PoW/PoS,-0.025168,-0.357064,-0.291282,EliteCoin
8,1.792718e+07,2.100000e+07,SHA-256,PoW,-0.090224,-0.510551,0.118172,Bitcoin
9,1.076842e+08,0.000000e+00,Ethash,PoW,-0.145378,0.004776,-0.273160,Ethereum
...,...,...,...,...,...,...,...,...
517,2.500124e+06,2.500124e+06,X13,PoS,0.118828,-0.347460,0.178812,RoyalCoin
523,1.000000e+08,1.000000e+08,X14,PoW/PoS,-0.077420,-0.100404,-0.171235,GanjaCoin V2
524,1.781868e+07,3.010000e+08,PoS,PoS,0.459081,-0.172178,0.669078,TeamUP
526,1.082163e+09,7.506000e+09,SHA-256D,PoW/PoS,-1.129224,3.222777,0.409304,LanaCoin


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [484]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df.hvplot.scatter(
   x="TotalCoinsMined_x",
   y="TotalCoinSupply_x",
   hover_cols=["CoinName"]
)

:Scatter   [TotalCoinsMined_x]   (TotalCoinSupply_x,CoinName)

#### Table of Tradable Cryptocurrencies

In [485]:
# Table with tradable cryptos
clustered_df.hvplot.table()

:Table   [TotalCoinsMined_x,TotalCoinSupply_x,Algorithm,ProofType,PC 1,PC 2,PC 3,CoinName]

In [486]:
# Print the total number of tradable cryptocurrencies
clustered_df['CoinName'].value_counts()

42 Coin          1
EverGreenCoin    1
Droidz           1
BowsCoin         1
Squall Coin      1
                ..
PhoenixCoin      1
Reddcoin         1
RonPaulCoin      1
StableCoin       1
Elementrem       1
Name: CoinName, Length: 205, dtype: int64